<table class="sprawko-header" style="width: 90%; border-style: hidden; font-size: 22px; line-height: 1.2em;">
<tr style="border-style: hidden;">
    <td style="width: 50%; border-style: hidden;">Informatyka, studia dzienne, II st.</td>
    <td style="width: 50%; text-align: right; border-style: hidden;">semestr II</td>
</tr>
<tr style="border-style: hidden;">
    <td style="width: 50%; border-style: hidden;">
        <b>Współczesne technologie programowania</b><br> Prowadzący: Kamil Stokfiszewski
    </td>
    <td style="width: 50%; text-align: right; border-style: hidden;">
        2016/2017<br> wtorek, 10:15
    </td>
</tr>
<tr style="border-style: hidden;">
    <td colspan="2">
        <ul style="width: 100%; text-align: center;  list-style: none;">
            <li>Daniel Pęczek 207585</li>
            <li>Michał Sośnicki 207597</li>
        </ul>
    </td>
</tr>
</table>

# Ćwiczenie 4: Symulacja zjawisk fizycznych na procesorach graficznych. . 

## Wprowadzenie
Celem zadania było przygotowanie symulacji zjawiska fizycznego przy wykorzystaniu procesorów graficznych. Wybraliśmy wariant z symulacją N-ciał fizycznych w środowisku grawitacyjnym. Głównym celem symulacji było zapewnienie, aby w każdej iteracji symulacji następowały po sobie cykle aktualizacji prędkości i pozycji ciała w przestrzeni $R^3$

## Realizacja

Program wykonujący symulację zrealizowano w języku `C++` z wykorzystaniem platformy `CUDA`. Na wejściu program przyjmuje osiem parametrów: liczbę ciał, ilość iteracji w symulacji, różnicę czasu między kolejnymi iteracjami symulacji, ilość iteracji, dolny i górny kraniec przedziału, z którego losowane są masy, szerokość przedziału, z którego losowane są położenia początkowe i taki sam parametr dla prędkości początkowych. Następne wyświetlane są kolejne pozycje obiektów.

Masa, pozycja i prędkość początkowa każdego ciała jest liczbą zmiennopozycyjną pojedynczej precyzji. Masy są losowane
z przedziału $[-arg_5, arg_6]$, położenie z $[-arg_7, arg_7]$, prędkości początkowe z $[-arg_8, arg_8]$, gdzie $arg_i$ to $i$ argument wywołania programu. 

Obliczenia na procesorze graficznym zostały zrealizowane przy pomocy dwóch kerneli - jeden odpowiedzialny za aktualizację prędkości, drugi za aktualizację pozycji w oparciu o nową prędkość - dzięki temu możemy wykorzystywać wiele ciał działających na kilku blokach, jako, że funkcja `_synchthreads()` synchronizuje jedynie te wątki, które znajdują się w danym bloku.

Do obliczania ilości wykorzystywanych wątków i bloków w kernelu wykorzystujemy funkcję `cudaOccupancyMaxPotentialBlockSize` na podstawie której obliczany jest potencjalny rozmiar bloku dla danego jądra - dzięki temu nie musimy ustawiać tych wartości ręcznie.

Analizę czasu wykonywania i wykresy wykonano w środowisku `Jupyter` w języku `Python` z bibliotekami `numpy` oraz `matplotlib`. Skrypty pozyskują wyniki poprzez uruchamianie opisanego wcześniej programu i analizę jego wyjścia. `Jupyter` umożliwia wzbogacenie kodu o dobrze prezentujące się wizualnie bloki `markdown`, więc ta część rozwiązania stanowi zarazem sprawozdanie z zadania.

Testy przeprowadzono na komputerze z kartą graficzną `NVIDIA GeForce GTS 450` (architektura `Fermi`) i procesorem `Intel Core i5-760 2.80 GHz`.

In [1]:
import functools as fn
import math
import re
import subprocess as sp

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from IPython.display import display

In [2]:
def run_get_stdout(name, *args):
    subprocess_args = [name] + list(map(str, args))
    output = sp.run(subprocess_args, stdout=sp.PIPE).stdout
    return output.decode('utf-8')

BLOCK_REGEX = re.compile(r"Block: (\d+) (\d+) (\d+)")
THREAD_REGEX = re.compile(r"Thread: (\d+) (\d+) (\d+)")
MASS_REGEX = re.compile(r"Body (\d+) mass: ([-\d\.]+)")
TIME_REGEX = re.compile(r"Time: ([-\d\.]+) s")
POSITION_REGEX = re.compile(r"Body (\d+) position: ([-\d\.]+) ([-\d\.]+) ([-\d\.]+)")

def parse_gravity(output, body_count):
    lines = output.splitlines()
    
    block_counts = map(int, BLOCK_REGEX.fullmatch(lines[0]).group(1, 2, 3))
    thread_counts = map(int, THREAD_REGEX.fullmatch(lines[1]).group(1, 2, 3))
    
    masses = np.zeros(body_count)
    for line in lines[2:2 + body_count]:
        mass_match = MASS_REGEX.fullmatch(line)
        index = int(mass_match.group(1))
        mass = float(mass_match.group(2))
        masses[index] = mass
    
    position_history = []
    positions =  None
    for line in lines[2 + body_count:-1]:
        time_match = TIME_REGEX.fullmatch(line)
        if time_match is not None:
            time = float(time_match.group(1))
            if positions is not None:
                position_history.append(positions)
            positions = np.zeros((body_count, 3))
        else:
            position_match = POSITION_REGEX.fullmatch(line)
            index = int(position_match.group(1))
            positions[index, :] = list(map(float, position_match.group(2, 3, 4)))
    return (tuple(block_counts), tuple(thread_counts), masses, np.array(position_history), lines)

def run_gravity(body_count, max_iterations, time_step, log_iterations, mass_min, mass_max, init_pos, init_v):
    output = run_get_stdout('../Zadanie4/main.out', 
                            body_count, max_iterations, time_step, log_iterations,
                            mass_min, mass_max, init_pos, init_v)
    return parse_gravity(output, body_count)

In [3]:
import base64
from IPython.display import HTML
from matplotlib import animation
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = base64.b64encode(video).decode('utf-8')
    
    return VIDEO_TAG.format(anim._encoded_video)

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

def animate_array(array, lims=None, interval=50):
    fig = plt.figure()
    if lims is None:
        xlim=(array[:, :, 0].min(), array[:, :, 0].max())
        ylim=(array[:, :, 1].min(), array[:, :, 1].max())
    else:
        xlim, ylim = lims
    ax = plt.axes(xlim=xlim, ylim=ylim)
    plot = ax.scatter([], [])

    def animate(i):
        plot.set_offsets(array[i, :, 0:2])
        return plot,

    anim = animation.FuncAnimation(fig, animate, frames=array.shape[0], interval=interval, blit=True)
    return display_animation(anim)

In [20]:
block_counts, thread_counts, masses, body_positions, log = run_gravity(100, 50000, 0.01, 100, 10, 50, 50, 1)

In [23]:
# print('\n'.join(log))

In [22]:
animate_array(body_positions, lims=((-200, 200), (-200, 200)))

## Wnioski

Architektura kart graficznych pozwala na stosunkową prostą implementację pozwalającą na szybkie, równoległe obliczenia, dzięki czemu pozwala na efektywną prezentację zagadnień związanych z symulacjami zjawisk fizycznych - dodatkowe wykorzystanie wizualizacji w OpenGL mogłoby odbywać się bezpośrednio z poziomu karty graficznej, przez co oszczędzony zostałby czas jaki byłby potrzebny na przesłanie tych danych z powrotem do pamięci komputera.